In [1]:
import json
import codecs
from outils import read, keys, load_cn_json, dump_cn_json, 中转数, 数转中
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle

def generate_rgb_colors(colormap_name, k, min_brightness=0.3):
    # Get the colormap from matplotlib
    cmap = plt.get_cmap(colormap_name)
    
    # Generate `k` evenly spaced values from 0 to 1
    norm = plt.Normalize(vmin=0, vmax=k-1)
    scalar_map = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    indices = np.arange(k)
    colors = scalar_map.to_rgba(indices)
    
    # Convert RGBA colors to RGB integers between 0 and 255
    rgb_colors = []
    for color in colors:
        brightness = 0.2126 * color[0] + 0.7152 * color[1] + 0.0722 * color[2]  # Luminance formula
        if brightness < min_brightness:
            rgb_colors.append((int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)))
    
    # If not enough colors are below the brightness threshold, reduce the threshold
    while len(rgb_colors) < k and min_brightness < 1:
        min_brightness += 0.05
        for color in colors:
            brightness = 0.2126 * color[0] + 0.7152 * color[1] + 0.0722 * color[2]
            if brightness < min_brightness and (int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)) not in rgb_colors:
                rgb_colors.append((int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)))
        rgb_colors = rgb_colors[:k]  # Ensure only `k` colors are returned
    
    return rgb_colors


def sort_dict_with(dic, key="grade"):
    pox = {}
    keys = []
    for k, v in dic:
        if v[key] not in pox:
            pox[key] = []
            keys.append(key)
        pox[key].append((k, v))
    
    out = []
    for key in sorted(keys):
        out.extend(pox[key])
    return out

In [686]:
def wrap(s, wrapper="{}", keep_wrapper=False):
    if s:
        return wrapper[0] + s + wrapper[-1]
    if keep_wrapper:
        return wrapper
    return ""

def make_params(params, wrapper="[]", sep=","):
    return wrap(sep.join(params), wrapper)

def wrap_env(name, content, params=[], param_wrapper="[]", param_sep=","):
    out = r"\begin" + wrap(name) + make_params(params, wrapper=param_wrapper, sep=param_sep) + "\n"
    lines = content[:-1].split("\n")  # presume content ends with \n
    for line in lines:
        out += "    " + line + "\n"
    out += r"\end" + wrap(name) + "\n"
    return out

def wrap_method(method, content="", wrapper="{}", keep_wrapper=True, params=[], param_wrapper="[]", param_sep=","):
    return '\\' + method + make_params(params, wrapper=param_wrapper, sep=param_sep) + wrap(content, wrapper=wrapper, keep_wrapper=keep_wrapper)
    
# print(wrap_env("center", "faowiefjawefijafo\naofiagjaoeija\nfai\n", params=["2pt", "tje"]))
# print(wrap_method("textbf"))

def zihao(n):
    return wrap_method("zihao", str(n))

def make_ctex_env(document_class="ctexbook", document_class_params=("12pt", "UTF-8","openany"), packages={"ctex": [], "titlesec": []}, mainfont="Arial", lineskip="4pt", parskip="10pt", title="标题", author="", date=False, toc=True):
    """make header and footer for ctexbook environment. 
    header
    1. documentclass and parameters 
    2. packages
    3. geometry and fonts
    4. package setups
    5. global typesettings
    6. begin document
    footer
    1. end document
    """
    # ## header ##

    # document class
    header = r"\documentclass"+ make_params(document_class_params) + wrap(document_class) + "\n"
    
    # packages
    packages_str = ""
    for name in packages:
        # print(package)
        package_declarations = ""
        if "declarations" in packages[name]:
            package_declarations = make_params(packages[name]['declarations'])
        packages_str += r"\usepackage" + package_declarations + wrap(name) + "\n"
    # print(packages_str)
    header += packages_str + "\n"

    # geometry <-- geometry package
    if "geometry" in packages:
        geometry = packages["geometry"]
        paper_type = geometry["paper_size"]
        paddings = geometry["paddings"]
        left = paddings["left"]
        right = paddings["right"]
        top = paddings["top"]
        bottom = paddings["bottom"]
        header += wrap_method("geometry", f"{paper_type}paper,left={left},right={right},top={top},bottom={bottom}") + "\n"
    
    # fonts
    header += wrap_method("setmainfont", mainfont) + "\n"
    header += r"\setCJKmainfont[BoldFont=STZhongsong]{汉字之美仿宋GBK 免费}" + "\n"
    header += r"\xeCJKDeclareCharClass{CJK}{`0 -> `9}" + "\n"  # apply CJK font to numbers
    header += r"\xeCJKsetup{AllowBreakBetweenPuncts=true}" + "\n"  # line alignment

    if "footmisc" in packages:
        footnote_settings_content = "".join(["{\ding{"+str(192+i)+"}}" for i in range(10)])
        footnote_settings = wrap_method("DefineFNsymbols", footnote_settings_content, params=["circled"], param_wrapper="{}")
        header += footnote_settings + "\n"
        header += wrap_method("setfnsymbol", "circled") + "\n"

    # package setups
    # xpinyin
    if "xpinyin" in packages:
        pyr = packages['xpinyin']['ratio']  # size ratio
        vsep = packages['xpinyin']['vsep']  # vertical gap
        vsep_str = "vsep={" + vsep + "}"
        hsep = packages['xpinyin']['hsep']  # horizontal gap
        hsep_str = "hsep={" + f"{hsep} plus {hsep}" + "}"
        header += wrap_method("xpinyinsetup", f"ratio={pyr},{hsep_str},{vsep_str}") + "\n"  # pinyin settings

    # package setups
    # xcolor
    if "xcolor" in packages:
        defcolor_str = ""
        for key, (r, g, b) in packages["xcolor"]["defined_colors"].items():
            rgb_plate = f"{r},{g},{b}"
            defcolor_str += wrap_method("definecolor", key) + r"{RGB}{" + rgb_plate + r"}" + "\n"
        header += defcolor_str + "\n"

    # global typesettings
    # title format
    header += r"\titleformat{\chapter}{\zihao{-1}\bfseries}{ }{16pt}{}" + "\n"
    header += r"\titleformat{\section}{\zihao{-2}\bfseries}{ }{0pt}{}" + "\n"
    header += r"\title" + wrap(r"\zihao{0} \bfseries " + title) + "\n"
    # line and paragraph skips
    header += r"\setlength{\lineskip}{" + lineskip + "}\n"  # skip length after line
    header += r"\setlength{\parskip}{" + parskip + "}\n"  # extra skip for paragraphs 
    # front page format
    if author:  # author format
        header += r"\author{\zihao{2} \texttt" + wrap(author) + "}\n"
    else:
        header += r"\author{}" + "\n"
    if date:  # date format
        header += r"\date{\bfseries\today}" + "\n"
    else:
        header += r"\date{}" + "\n"
    
    # begin document
    header += r"\begin" + wrap("document") + "\n"
    header += r"\maketitle" + "\n"
    if toc:
        header += r"\tableofcontents" + "\n"
    header += r"\newpage" + "\n"
    
    # ## footer ##

    # end document
    footer = r"\end" + wrap("document") + "\n"
    return header, footer

def package_update_xcolor(packages, texts):
    xcolor = packages["xcolor"]
    xcolor["defined_colors"] = {}
    for _, text in texts.items():
        if "character_colors" in text:
            for key, val in text["character_colors"].items():
                xcolor["defined_colors"][key] = val
    packages["xcolor"] = xcolor

In [3]:
# 打印页面设置：纸号，页边距等
geometry = {}
geometry["paper_size"] = "a5"  # 使用A5纸
paddings = {}  # 页边距
paddings["left"] = "1.4cm"
paddings["right"] = "1.4cm"
paddings["top"] = "2.4cm"
paddings["bottom"] = "2.4cm"
geometry["paddings"] = paddings

# 拼音设置： xpinyin宏包
pinyin = {}
pinyin["ratio"] = "0.5"
pinyin["hsep"] = ".6em"
pinyin["vsep"] = "1em"

In [4]:
# unit test
packages = {}
packages['xpinyin'] = pinyin
pyr = packages['xpinyin']['ratio']  # size ratio
vsep = packages['xpinyin']['vsep']  # vertical gap
vsep_str = "vsep={" + vsep + "}"
hsep = packages['xpinyin']['hsep']  # horizontal gap
hsep_str = "hsep={" + f"{hsep} plus {hsep}" + "}"
wrap_method("xpinyinsetup", f"ratio={pyr},{hsep_str},{vsep_str}") + "\n"  # pinyin settings

'\\xpinyinsetup{ratio=0.5,hsep={.6em plus .6em},vsep={1em}}\n'

## 古诗转换

### 古诗tex转json

In [3]:
# path_tex = "../src/小学/古诗.tex"

# with open(path2, 'r', encoding="utf-8") as f:
#     lines = f.readlines()

### 古诗json转tex

In [5]:
path_json = "../src/小学/古诗.json"

shis = load_cn_json(path_json)

In [6]:
# with codecs.open("../src/小学/唐诗三百首.json", "r", encoding="utf-8") as f:
#     tangshi300 = json.load(f)

tangshi300 = load_cn_json("../src/小学/唐诗三百首.json")

authors = {}

for shi in tangshi300:
    if shi["author"] not in authors:
        authors[shi["author"]] = "唐代"

In [7]:
def shi_to_tex_str(shi, print_genre=False, authors={}, typesettings={"vspaces": {"after_title": 8, "after_author": 6, "after_content": 6}}):
    # convert structured shi to string ready to use in tex
    out = r"\section{" + shi["title"] + "}\n\n"
    content = ""
    # title = wrap_method("textbf", zihao(3) + " " + shi["title"]) + "\n\n"

    # if print_genre:
    #     title = shi["genre"] + "：" + title
    # content += title
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_title']}pt") + "\n\n"
    author_str = ""
    if shi["author"]:
        author = shi["author"]
        if author in authors:
            author_str += "〔唐代：" + author + "〕\n\n"
        else:
            author_str += "〔" + author + "〕\n\n"
    else:
        author_str += "〔作者不详〕\n\n"
    content += wrap_env("normalsize", "\n" + author_str) + "\n"
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_author']}pt") + "\n\n"
    content += wrap_env("large", "\n" + "\n\n".join([wrap_method("xpinyin*", line) for line in shi["content"]]) + "\n\n") + "\n"
    content = wrap_env("center", content) + "\n"
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_content']}pt") + "\n\n"
    out += content
    return out

for title, shi in shis.items():
    s = shi_to_tex_str(shi, print_genre=True, authors=authors)
    break

print(s)

\section{江南}

\begin{center}
    \vspace{8pt}
    
    \begin{normalsize}
        
        〔唐代：无名氏〕
        
    \end{normalsize}
    
    \vspace{6pt}
    
    \begin{large}
        
        \xpinyin*{江南可采莲，}
        
        \xpinyin*{莲叶何田田，}
        
        \xpinyin*{鱼戏莲叶间。}
        
        \xpinyin*{鱼戏莲叶东，}
        
        \xpinyin*{鱼戏莲叶西，}
        
        \xpinyin*{鱼戏莲叶南，}
        
        \xpinyin*{鱼戏莲叶北。}
        
    \end{large}
    
\end{center}

\vspace{6pt}




In [7]:
shis["春晓"]

{'level': '第二层',
 'content': ['春眠不觉晓，处处闻啼鸟。', '夜来风雨声，花落知多少。'],
 'author': '孟浩然',
 'genre': '五言绝句',
 'title': '春晓'}

In [8]:
# 打印小学古诗（分层）
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["xpinyin"]['ratio'] = "0.44"
packages["xpinyin"]['hsep'] = ".6em"
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
title = "小学语文古诗集"
lineskip = "24pt"
parskip = "6pt"
header, footer = make_ctex_env(packages=packages, title=title, parskip=parskip, lineskip=lineskip)

# 分层
shi_by_level = {}
levels = []
for i in range(10):
    levels.append(f"第{数转中[i+1]}层") 
levels.append("其他")
# print(levels)

typesettings = {"vspaces": {"after_title": 10, "after_author": 8, "after_content": 8}}

for title, shi in shis.items():
    level = shi["level"]
    if level not in shi_by_level:
        shi_by_level[level] = {}
    shi_by_level[level][title] = shi

with open("古诗集.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for level in levels:
        f.write(r"\chapter" + wrap(level) + "\n\n")
        for title, shi in shi_by_level[level].items():
            f.write(shi_to_tex_str(shi, typesettings=typesettings) + "\n")
    f.write(footer)

## 小学课文

In [133]:
texts = load_cn_json("../src/小学/阅读课文.json")
print(texts["镜泊湖奇观"]["endnotes"])

['〔弭〕平息，止。', '〔袪〕驱逐，去除。', '〔峭拔〕（山）高而陡。', '〔峙〕直立，耸立。', '〔斛〕容量单位。一斛十斗。']


In [674]:
def read_text(path, format="散文", max_name_len=13):
    lines = read(path)
    out = {}
    title = ""
    author = ""
    grade = 0
    footnotes = []
    endnotes = []
    vocabulary = []
    remarks = []
    content = []
    out["format"] = format
    out["genre"] = [format]
    # return lines
    if format in ("散文", "书信", "小说", "剧本"):
        for line in lines:
            line0 = line.strip()
            if line0:
                if not title:
                    title = line0
                elif grade < 1 and line.startswith("年级："):
                    grade = int(line0[3:])
                elif not author and line.startswith("作者：") and len(line0) < max_name_len:
                    author = line0[3:]
                elif line.startswith("备注："):
                    remarks.append(line0[3:])
                elif line.startswith("注释："):
                    footnotes.append(line0[3:])
                elif line.startswith("脚注："):
                    footnotes.append(line0[3:])
                elif line.startswith("尾注："):
                    endnotes.append(line0[3:])
                elif line.startswith("词汇："):
                    vocabulary.extend(line0[3:].split())
                else:
                    content.append(line0)
    elif format == "诗歌":
        para = []
        for line in lines:
            line0 = line.strip()
            if line0:
                if not title:
                    title = line0
                elif grade < 1 and line.startswith("年级："):
                    grade = int(line0[3:])
                elif not author and line.startswith("作者：") and len(line0) < max_name_len:
                    author = line0[3:]
                elif line.startswith("备注："):
                    remarks.append(line0[3:])
                elif line.startswith("注释："):
                    footnotes.append(line0[3:])
                elif line.startswith("脚注："):
                    footnotes.append(line0[3:])
                elif line.startswith("尾注："):
                    endnotes.append(line0[3:])
                elif line.startswith("词汇："):
                    vocabulary.extend(line0[3:].split())
                else:
                    para.append(line0)
            elif len(para):
                content.append(para)
                para = []
        if len(para):
            content.append(para)
    out["title"] = title
    out["author"] = author
    out["content"] = content
    out["remarks"] = remarks
    out["footnotes"] = footnotes
    out["endnotes"] = endnotes
    out["vocabulary"] = vocabulary
    if grade:
        out["grade"] = grade
    return title, out

text_format = "剧本"
title, content = read_text("草稿.tex", format=text_format)

content["genre"] = ["记叙文", "回忆"]

texts[title] = content
# dump_cn_json("../src/小学/阅读课文.json", texts)
# content

In [88]:
# for title, text in texts.items():
#     footnotes = []
#     endnotes = []
#     for note in text["footnotes"]:
#         parts = note.split("：")
#         word = parts[0]
#         footnotes.append("〔" + word + "〕" + "：".join(parts[1:]))
#         text["footnotes"] = footnotes
#     for note in text["endnotes"]:
#         parts = note.split("：")
#         word = parts[0]
#         endnotes.append("〔" + word + "〕" + "：".join(parts[1:]))
#         text["endnotes"] = endnotes


In [706]:
def shortstack(content, lcr="l"):
    # out = r"\linespread" + wrap(scale) + r" \selectfont" + "\n"
    out = "\n"
    lines = content[:-1].split("\n")  # presume content ends with \n
    for line in lines[:-1]:
        out += "    " + line + " \\\\\n"
    out += "    " + lines[-1] + "\n"
    out = wrap_method("shortstack", out, params=[lcr])
    return out

def text_content_to_tex_str(text, verbose=0, verseprop=0.5):
    """convert a the content of a text object to text string ready for tex.
    the format varies by genre:
    散文
    书信
    诗歌
    剧本
    """ 
    content = text["content"]
    footnotes = text["footnotes"]
    format = text["format"]
    out = ""
    if format in ("散文", "小说",):
        out = "\n\n".join(content) + "\n"
    elif  format == "书信":
        if verbose and not content[0].endswith("："):
            print("错误：第一行不是抬头")
            return "格式错误\n"
        out = r"\noindent " + content[0] + "\n\n" + wrap_method("vspace", "24pt") + "\n\n"
        toright = False
        toright_content = ""
        for line in content[1:]:
            if line:
                if toright:
                    toright_content += line + "\n\n"
                else:
                    out += line + "\n\n"
            else:
                toright = True
        out += wrap_method("vspace", "36pt") + "\n\n"
        out += wrap_env("flushright", toright_content) + "\n\n"
    elif  format == "诗歌":
        if not isinstance(content[0], list):
            content = [content]
        
        lineskip = " \\\\\n"
        # parskip = "\n" + wrap_method("vspace", "4pt") + "\n\n"
        parskip = "\n\n"
        out = parskip.join([wrap_env("verse", lineskip.join(par) + "\n", params=[str(verseprop)+"\\linewidth"]) for par in content])
    elif format == "剧本":
        name_set = text["characters"]
        for line in content:
            if line.startswith("\\item["):
                name = line.split("]")[0][6:]
                colored_name = r"{\color{" + name_set[name] + r"} " + name + r"}"
                out += "\\item[" + colored_name + "]" + "]".join(line.split("]")[1:])
            elif line.startswith("$"):
                colored_line = line
                for name in name_set:
                    colored_line = colored_line.replace(name, r"{\color{" + name_set[name] + r"} " + name + r"}")
                out += colored_line
            else:
                out += line
            out += "\n\n"
    if len(footnotes):
        if "footnote" in out:  # add footnotes when the position is provided
            for i, note in enumerate(footnotes):
                out = out.replace("footnote{"+str(i+1)+"}", "footnote{" + note + "}")
                # out = ssub(r"footnote\{(\d+?)\}", "footnote{" + note + "}", out, 1)
        else:
            note_dict = {}
            keybase = "b"
            i = 0
            for note in footnotes:
                word = ""
                if note.startswith("〔"):
                    word = note.split("〕")[0][1:]
                    key = keybase + str(i+1)
                    note_dict[key] = note
                    i += 1
                elif "〕" in note:
                    key = note.split("〕")[0].split("〔")[0]
                    note_dict[key] = "".join(note.split(key)[1:])
                # print(word)
                if word:  # find where to insert footnote and mark
                    nfin = out.find(word) + len(word)
                    out = out[:nfin] + r"\apost{" + key + "}" + out[nfin:]
            for key in note_dict:
                out = out.replace("apost{"+key+"}", "footnote{" + note_dict[key] + "}")

    return out

def endnotes_to_str(endnotes, verbose=0, pinyin=False):
    out = ""
    notes = ""
    for note in endnotes:
        if pinyin and note.startswith("〔"):  # add pinyin
            suite = note[1:].split("〕")
            notes += "\item " + note[0] + r"\xpinyin*{" + suite[0] + r"}〕" + "〕".join(suite[1:]) + "\n"
        else:
            notes += "\item " + note + "\n"
    if notes:
        out = r"\newpage" + "\n\n" + r"\textbf{注释}：" + "\n\n" + r"\vspace{-1em}" + "\n\n"
        out += wrap_env("itemize", r"\setlength\itemsep{-0.2em}" + "\n" + notes)
    return out

def text_to_tex_str(text, typesettings={"font": {"title": {"size": 2}, "plaintext": {"size": "normalsize"}}, "vspaces": {"after_title": 12, "after_author": 6, "after_content": 6}}):
    """convert a text object to text string ready for tex
    """
    out = ""
    content = ""
    # title_fontsize = typesettings["font"]["title"]["size"]
    # title = wrap_method("textbf", zihao(title_fontsize) + " " + text["title"]) + "\n"
    title = wrap_method("chapter", text["title"]) + "\n\n"
    content += title
    # content = wrap_env("center", content) + "\n"
    # content += wrap_method("vspace", f"{typesettings['vspaces']['after_title']}pt") + "\n\n"
    content += wrap_env(typesettings["font"]["plaintext"]["size"], "\n" + text_content_to_tex_str(text) + "\n")
    # content += wrap_method("vspace", f"{typesettings['vspaces']['after_content']}pt") + "\n\n"
    out += content + "\n\n"
    # out += wrap_method("newpage", keep_wrapper=False) + "\n\n"
    out += endnotes_to_str(text["endnotes"])
    return out

title = "有的人"
print(text_to_tex_str(texts[title]) + "...")

\chapter{有的人}

\begin{normalsize}
    
    \begin{verse}[0.5\linewidth]
        有的人活着，他已经死了； \\
        有的人死了，他还活着。 \\
        有的人骑在人民头上：“呵，我多伟大！” \\
        有的人俯下身子给人民当牛马。 \\
        有的人把名字刻入石头，想“不朽”； \\
        有的人情愿作野草，等着地下的火烧。 \\
        有的人，他活着别人就不能活； \\
        有的人，他活着为了多数人更好地活。
    \end{verse}
    
    
    \begin{verse}[0.5\linewidth]
        骑在人民头上的，人民把他摔垮； \\
        给人民作牛马的，人民永远记住他！ \\
        把名字刻入石头的，名字比尸首烂得更早； \\
        只要春风吹到的地方，到处是青青的野草。 \\
        他活着别人就不能活的人，他的下场可以看到； \\
        他活着为了多数人更好地活的人，群众把他抬举得很高，很高。
    \end{verse}
    
\end{normalsize}


...


In [139]:
# 打印小学课文（每篇文章换页）
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["verse"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
lineskip = "24pt"
parskip = "6pt"

# for title, shi in shis.items():
#     level = shi["level"]
#     if level not in shi_by_level:
#         shi_by_level[level] = {}
#     shi_by_level[level][title] = shi

In [693]:
texts = load_cn_json("../src/小学/阅读课文.json")

booktitle = "小学语文课文集萃"
header, footer = make_ctex_env(packages=packages, title=booktitle, parskip=parskip, lineskip=lineskip)
typesettings = {}
typesettings["vspaces"] = {"after_title": 36, "after_author": 16, "after_content": 16}
typesettings["font"] = {"plaintext": {"size": "large"}}

with open("阅读课文.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for title, text in texts.items():
        f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
    f.write(footer)

## 初中课文

In [704]:
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["verse"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["enumitem"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
xcolor = {}
xcolor["declarations"] = ["table", "dvipsnames"]
packages["xcolor"] = xcolor
lineskip = "24pt"
parskip = "6pt"

booktitle = "初中语文课文集萃"
typesettings = {}
typesettings["vspaces"] = {"after_title": 36, "after_author": 16, "after_content": 16}
typesettings["font"] = {"plaintext": {"size": "normalsize"}}

In [675]:
texts2 = load_cn_json("../src/初中/阅读课文.json")

def set_char_colors(content, script_key, colormap_name="tab10", min_brightness=0.5):
    """Set the colors for the characters in a script.
    Inputs:
    content (list of str): the content of the script. With each line the speech of one character at a time beginning with \\item[***]. 
    script_key (str): the key of the script so as to generate color keys. The color keys start with the script key.
    colormap_name (str): name of the colormap to use. 
    min_brightness (float): minimal brightness requirement so that the colors don't get too light. smaller min means stricter (darker) colors.
    Output:
    name_set: set of character names with color key. dict: name --> key 
    color_set: set of colors by key. dict: key --> color
    """
    name_set = {}
    color_set = {}
    i = 0
    for line in content:
        if line.startswith("\\item["):
            name = line.split(']')[0][6:]
            if name not in name_set:
                name_key = script_key + "-" + str(i)
                name_set[name] = name_key
                # color_set[name_key] = 
                i += 1

    nchar = len(name_set)
    rgb_colors = generate_rgb_colors(colormap_name, nchar, min_brightness=min_brightness)
    shuffle(rgb_colors)
    i = 0
    for _, key in name_set.items():
        color_set[key] = rgb_colors[i]
        i += 1
    return name_set, color_set

text_format = "剧本"
title, content = read_text("草稿.tex", format=text_format)

content["genre"] = ["记叙文", "回忆"]

if text_format == "剧本":
    if title not in texts2:
        script_keys = []
        for _, text in texts2.items():
            if text["format"] == "剧本" and "key" in text:
                script_keys.append(int(text["key"].split("-")[1]))
        if len(script_keys):
            script_key = "script-" + str(max(script_keys) + 1)
        else:
            script_key = "script-1"
            
    else:
        script_key = texts2[title]["key"]
    name_set, color_set = set_char_colors(content["content"], script_key)
    
texts2[title] = content
texts2[title]["key"] = script_key
texts2[title]["characters"] = name_set
texts2[title]["character_colors"] = color_set

dump_cn_json("../src/初中/阅读课文.json", texts2)

In [660]:
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle

def generate_rgb_colors(colormap_name, k, min_brightness=0.3):
    # Get the colormap from matplotlib
    cmap = plt.get_cmap(colormap_name)
    
    # Generate `k` evenly spaced values from 0 to 1
    norm = plt.Normalize(vmin=0, vmax=k-1)
    scalar_map = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    indices = np.arange(k)
    colors = scalar_map.to_rgba(indices)
    
    # Convert RGBA colors to RGB integers between 0 and 255
    rgb_colors = []
    for color in colors:
        brightness = 0.2126 * color[0] + 0.7152 * color[1] + 0.0722 * color[2]  # Luminance formula
        if brightness < min_brightness:
            rgb_colors.append((int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)))
    
    # If not enough colors are below the brightness threshold, reduce the threshold
    while len(rgb_colors) < k and min_brightness < 1:
        min_brightness += 0.05
        for color in colors:
            brightness = 0.2126 * color[0] + 0.7152 * color[1] + 0.0722 * color[2]
            if brightness < min_brightness and (int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)) not in rgb_colors:
                rgb_colors.append((int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)))
        rgb_colors = rgb_colors[:k]  # Ensure only `k` colors are returned
    
    return rgb_colors

def set_char_colors(content, script_key, colormap_name="tab10", min_brightness=0.5):
    name_set = {}
    color_set = {}
    i = 0
    for line in content:
        if line.startswith("\\item["):
            name = line.split(']')[0][6:]
            if name not in name_set:
                name_key = script_key + "-" + str(i)
                name_set[name] = name_key
                # color_set[name_key] = 
                i += 1

    # char_set = {v:k for k, v in name_set.items()}
    nchar = len(name_set)
    rgb_colors = generate_rgb_colors(colormap_name, nchar, min_brightness=min_brightness)
    shuffle(rgb_colors)
    i = 0
    for _, key in name_set.items():
        color_set[key] = rgb_colors[i]
        i += 1
    return name_set, color_set

script_key = "script-1"
name_set, color_set = set_char_colors(content["content"], script_key)
# char_set = {v:k for k, v in name_set.items()}
print("Sample text output with the generated colors:")
for name in name_set:
    key = name_set[name]
    color = color_set[key]
    hex_color = '#%02x%02x%02x' % color
    print(f"\033[38;2;{color[0]};{color[1]};{color[2]}m{name} \033[0m - {hex_color}")


Sample text output with the generated colors:
公爵  - #7f7f7f
安东尼奥  - #bcbd22
萨拉里诺  - #8c564b
夏洛克  - #e377c2
巴萨尼奥  - #2ca02c
尼莉莎  - #17becf
葛莱西安诺  - #1f77b4
书记  - #d62728
鲍西娅  - #ff7f0e


In [159]:
texts2 = load_cn_json("../src/初中/阅读课文.json")

In [ ]:
title = "阿长与山海经"
print(text_to_tex_str(texts2[title]) + "...")

In [682]:
package_update_xcolor(packages, texts2)

In [685]:
defcolor_str = ""
for key, (r, g, b) in packages["xcolor"]["defined_colors"].items():
    rgb_plate = f"{r},{g},{b}"
    defcolor_str += wrap_method("definecolor", key) + r"{RGB}{" + rgb_plate + r"}" + "\n"

print(defcolor_str)

\definecolor{script-1-0}{RGB}{23,190,207}
\definecolor{script-1-1}{RGB}{127,127,127}
\definecolor{script-1-2}{RGB}{31,119,180}
\definecolor{script-1-3}{RGB}{227,119,194}
\definecolor{script-1-4}{RGB}{188,189,34}
\definecolor{script-1-5}{RGB}{44,160,44}
\definecolor{script-1-6}{RGB}{214,39,40}
\definecolor{script-1-7}{RGB}{140,86,75}
\definecolor{script-1-8}{RGB}{255,127,14}



In [689]:
texts2["威尼斯商人"]

{'format': '剧本',
 'genre': ['记叙文', '回忆'],
 'title': '威尼斯商人',
 'author': '威廉·莎士比亚',
 'content': ['第四幕 第一场',
  '威尼斯，法庭。',
  '$\\triangleright$公爵、安东尼奥、巴萨尼奥、葛莱西安诺、萨拉里诺、萨莱尼奥、众绅士及其余人等齐上。',
  '\\begin{description}[itemsep=1ex,leftmargin=2cm]',
  '\\item[公爵]安东尼奥有没有来？',
  '\\item[安东尼奥]有，殿下。',
  '\\item[公爵]我很为你不快乐；你是来跟一个心如铁石的对手当庭质对，一个不懂得怜悯、没有一丝慈悲心的不近人情的恶汉。',
  '\\item[安东尼奥]听说殿下曾经用尽力量劝他不要过为已甚，可是他一味坚执，不肯略作让步。既然没有合法的手段可以使我脱离他的怨毒的掌握，我只有用默忍迎受他的愤怒，安心等待着他的残暴的处置。',
  '\\item[公爵]来人，传那犹太人到庭。',
  '\\item[萨拉里诺]他在门口等着；他来了，殿下。',
  '\\end{description}',
  '$\\triangleright$夏洛克上。',
  '\\begin{description}[itemsep=1ex,leftmargin=2cm]',
  '\\item[公爵]大家让开些，让他站在我的面前。夏洛克，人家都以为——我也是这样想——你不过故意装出这一副凶恶的姿态，到了最后关头，就会显出你的仁慈恻隐来，比你现在这种表面上的残酷更加出人意料；现在你虽然坚持着照约处罚，一定要从这个不幸的商人身上割下一磅肉来，到了那时候，你不但愿意放弃这一种处罚，而且因为受到良心上的感动，说不定还会豁免他一部分的欠款。你看他最近接连遭逢的巨大损失，足以使无论怎样富有的商人倾家荡产，即使铁石一样的心肠，从来不知道人类同情的野蛮人，也不能不对他的境遇发生怜悯。犹太人，我们都在等候你一句温和的回答。',
  '\\item[夏洛克]我的意思已经向殿下告禀过了；我也已经指着我们的圣安息日起誓，一定要照约执行处罚；要是殿下不准许我的请求，那就是蔑视宪章，我要到京城里去上告，要求撤销贵邦的特权。您要是问我为什么不愿接受三千块钱

In [844]:
texts2 = load_cn_json("../src/初中/阅读课文.json")

package_update_xcolor(packages, texts2)
header, footer = make_ctex_env(packages=packages, title=booktitle, parskip=parskip, lineskip=lineskip)
with open("初中现代文阅读课文.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for title, text in texts2.items():
        f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
    f.write(footer)

## 下载

In [144]:
import requests

s = "1c73b348-e8b6-47d6-84b0-6dbacbe28268"
url = f"https://r3-ndr.ykt.cbern.com.cn/edu_product/esp/assets/{s}.pkg/pdf.pdf"

# r = requests.get(url)
# open('sample.pdf', 'wb').write(r.content)

In [5]:
url = r"https://basic.smartedu.cn/tchMaterial"

r = requests.get(url)

In [17]:
src = "D:/Documents/Projects/yuwen/src/小学/"

with open(src + "urlcode.txt", "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    idx = line.rstrip("\n")
    out_filename = src + f"book_{i+1:02d}.pdf"
    print(f"Saving to {out_filename}")
    url = f"https://r3-ndr.ykt.cbern.com.cn/edu_product/esp/assets/{idx}.pkg/pdf.pdf"
    r = requests.get(url)
    open(out_filename, 'wb').write(r.content)

Saving to D:/Documents/Projects/yuwen/src/小学/book_01.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_02.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_03.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_04.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_05.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_06.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_07.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_08.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_09.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_10.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_11.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_12.pdf
Saving to D:/Documents/Projects/yuwen/src/小学/book_13.pdf


In [146]:
import os

src = "D:/Documents/data/textbooks/"
urls = os.path.join(src, "urls")

# for x in ("xiaoxueshuxue", "zhongxueyuwen", "zhongxueshuxue"):
for x in ("xiaoxueshuxue", ):
    print(f"Dealing with: {x}.")
    with open(os.path.join(urls, f"{x}.txt"), "r") as f:
        lines = f.readlines()
    # print(lines)
    out_dir = os.path.join(src, x)
    print(out_dir)

    if not os.path.exists(out_dir):
        print(out_dir, "doesn\'t exist.")
        os.mkdir(out_dir)
    
    i = 0
    for line in lines:
        if len(line) > 5:
            if line.startswith(">"):
                title = line[1:-1]
                i = 0
            else:
                i += 1
                idx = line.split("contentId=")[1].split("&")[0]
                print(idx)
                out_filename = os.path.join(out_dir, f"{title}_book_{i:02d}.pdf")
                print(f"Saving to {out_filename}")
                url = f"https://r3-ndr.ykt.cbern.com.cn/edu_product/esp/assets/{idx}.pkg/pdf.pdf"
                r = requests.get(url)
                open(out_filename, 'wb').write(r.content)

Dealing with: xiaoxueshuxue.
D:/Documents/data/textbooks/xiaoxueshuxue
D:/Documents/data/textbooks/xiaoxueshuxue doesn't exist.
c3e06fe4-c6b3-49cb-8727-4f8ff69bbfbc
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_01.pdf
6bf8ae7e-d987-40b4-8fb3-bbb98fcb50b5
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_02.pdf
8cfc5a2a-425c-4b9a-a97c-e78d4a4c1e3a
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_03.pdf
c1897b18-b302-4e8d-9fd4-40915c4b05c2
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_04.pdf
33c8d495-9862-4e19-aab9-61d2af08608a
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_05.pdf
8666a8bd-a0e7-49aa-ba07-bf419ceead24
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_06.pdf
654e3d1e-c995-4340-81c5-abd7881d835b
Saving to D:/Documents/data/textbooks/xiaoxueshuxue\renjiaoban_book_07.pdf
aa00ab9d-b343-4542-b16d-9c3900b3444b
Saving to D:/Documents/data/textbooks/xiaoxueshuxue

In [7]:
suanshi111 = []

for i in range(1, 10):
    for j in range(1, 10):
        if i + j <= 10:
            suanshi111.append(f"{i} + {j} = {i+j}")

import numpy as np

# help(np.random.permutation)
suanshi111 = np.random.permutation(suanshi111)

c = 0
pn = 3
out = []
hang = []
for s in suanshi111:
    hang.append(s)
    c += 1
    if c == pn:
        out.append("\quad \qquad & \quad \qquad".join(hang) + " \\\\")
        hang = []
        c = 0
if len(hang):
    out.append("\quad \quad &\qquad  \quad".join(hang) + " \\\\")
print("\n".join(out))
        

1 + 4 = 5\quad \qquad & \quad \qquad1 + 2 = 3\quad \qquad & \quad \qquad2 + 8 = 10 \\
7 + 2 = 9\quad \qquad & \quad \qquad4 + 2 = 6\quad \qquad & \quad \qquad3 + 5 = 8 \\
3 + 2 = 5\quad \qquad & \quad \qquad3 + 6 = 9\quad \qquad & \quad \qquad2 + 3 = 5 \\
5 + 5 = 10\quad \qquad & \quad \qquad6 + 3 = 9\quad \qquad & \quad \qquad3 + 4 = 7 \\
2 + 2 = 4\quad \qquad & \quad \qquad7 + 1 = 8\quad \qquad & \quad \qquad6 + 4 = 10 \\
1 + 8 = 9\quad \qquad & \quad \qquad5 + 4 = 9\quad \qquad & \quad \qquad1 + 3 = 4 \\
4 + 1 = 5\quad \qquad & \quad \qquad1 + 7 = 8\quad \qquad & \quad \qquad1 + 9 = 10 \\
1 + 1 = 2\quad \qquad & \quad \qquad5 + 3 = 8\quad \qquad & \quad \qquad3 + 3 = 6 \\
1 + 5 = 6\quad \qquad & \quad \qquad6 + 2 = 8\quad \qquad & \quad \qquad3 + 7 = 10 \\
2 + 5 = 7\quad \qquad & \quad \qquad2 + 7 = 9\quad \qquad & \quad \qquad4 + 3 = 7 \\
6 + 1 = 7\quad \qquad & \quad \qquad4 + 5 = 9\quad \qquad & \quad \qquad1 + 6 = 7 \\
2 + 4 = 6\quad \qquad & \quad \qquad5 + 1 = 6\quad \qquad & 

In [13]:
suanshi111ex = []

for i in range(1, 10):
    for j in range(1, 10):
        if i + j <= 10:
            suanshi111ex.append(f"\dottedsquare + {j} = {i+j} ")
            # suanshi111ex.append(f"{i} + \dottedsquare = {i+j} ")

suanshi111ex = np.random.permutation(suanshi111ex)

c = 0
pn = 4
nex = 20
out = []
hang = []
for s in suanshi111ex[:nex]:
    hang.append(s)
    c += 1
    if c == pn:
        out.append("\quad & \quad ".join(hang) + " \\\\")
        hang = []
        c = 0
if len(hang):
    out.append("\quad & \quad ".join(hang) + " \\\\")
print("\n".join(out))
        

\dottedsquare + 1 = 4 \quad & \quad \dottedsquare + 7 = 9 \quad & \quad \dottedsquare + 4 = 10 \quad & \quad \dottedsquare + 1 = 7  \\
\dottedsquare + 6 = 7 \quad & \quad \dottedsquare + 2 = 7 \quad & \quad \dottedsquare + 1 = 8 \quad & \quad \dottedsquare + 5 = 8  \\
\dottedsquare + 6 = 10 \quad & \quad \dottedsquare + 5 = 6 \quad & \quad \dottedsquare + 2 = 6 \quad & \quad \dottedsquare + 1 = 5  \\
\dottedsquare + 1 = 2 \quad & \quad \dottedsquare + 8 = 10 \quad & \quad \dottedsquare + 4 = 8 \quad & \quad \dottedsquare + 5 = 10  \\
\dottedsquare + 4 = 5 \quad & \quad \dottedsquare + 3 = 9 \quad & \quad \dottedsquare + 7 = 8 \quad & \quad \dottedsquare + 8 = 9  \\


In [51]:
bishi111ex = []

for i in range(1, 11):
    for j in range(1, 11):
        si = "\,\,"
        sj = "\,\,"
        if i < 10:
            si += "\,\;"
        if j < 10:
            sj += "\,\;"
        if i != j:
            bishi111ex.append(f"{i} {si} \\dottedsquare {sj} {j}")

bishi111ex = np.random.permutation(bishi111ex)

c = 0
pn = 4
nex = 28
out = []
hang = []
for s in bishi111ex[:nex]:
    hang.append(s)
    c += 1
    if c == pn:
        out.append("\qquad & \qquad ".join(hang) + " \\\\")
        hang = []
        c = 0
if len(hang):
    out.append("\qquad & \qquad ".join(hang) + " \\\\")
print("\n".join(out))

2 \,\,\,\; \dottedsquare \,\,\,\; 5\qquad & \qquad 9 \,\,\,\; \dottedsquare \,\,\,\; 3\qquad & \qquad 2 \,\,\,\; \dottedsquare \,\,\,\; 8\qquad & \qquad 6 \,\,\,\; \dottedsquare \,\,\,\; 1 \\
2 \,\,\,\; \dottedsquare \,\,\,\; 7\qquad & \qquad 6 \,\,\,\; \dottedsquare \,\,\,\; 3\qquad & \qquad 3 \,\,\,\; \dottedsquare \,\,\,\; 4\qquad & \qquad 3 \,\,\,\; \dottedsquare \,\,\,\; 2 \\
2 \,\,\,\; \dottedsquare \,\,\,\; 1\qquad & \qquad 7 \,\,\,\; \dottedsquare \,\,\,\; 3\qquad & \qquad 7 \,\,\,\; \dottedsquare \,\,\,\; 5\qquad & \qquad 3 \,\,\,\; \dottedsquare \,\,\,\; 8 \\
5 \,\,\,\; \dottedsquare \,\,\,\; 3\qquad & \qquad 1 \,\,\,\; \dottedsquare \,\, 10\qquad & \qquad 9 \,\,\,\; \dottedsquare \,\,\,\; 6\qquad & \qquad 6 \,\,\,\; \dottedsquare \,\,\,\; 2 \\
1 \,\,\,\; \dottedsquare \,\,\,\; 5\qquad & \qquad 3 \,\,\,\; \dottedsquare \,\,\,\; 7\qquad & \qquad 1 \,\,\,\; \dottedsquare \,\,\,\; 9\qquad & \qquad 1 \,\,\,\; \dottedsquare \,\,\,\; 3 \\
4 \,\,\,\; \dottedsquare \,\, 10\qquad & \q

In [49]:
bishi111 = []

for i in range(1, 11):
    for j in range(1, 11):
        si = "\,\,\,"
        sj = "\,\,\,"
        if i < 10:
            si += "\,\,"
        if j < 10:
            sj += "\,\,"
        if i < j:
            bishi111.append(f"{i} {si} < {sj} {j}")
        if i > j:
            bishi111.append(f"{i} {si} > {sj} {j}")

bishi111 = np.random.permutation(bishi111)

c = 0
pn = 5
out = []
hang = []
for s in bishi111:
    hang.append(s)
    c += 1
    if c == pn:
        out.append("\quad & \quad ".join(hang) + " \\\\")
        hang = []
        c = 0
if len(hang):
    out.append("\quad & \quad ".join(hang) + " \\\\")
print("\n".join(out))

7 \,\,\,\,\, < \,\,\,\,\, 9\quad & \quad 9 \,\,\,\,\, > \,\,\,\,\, 3\quad & \quad 2 \,\,\,\,\, < \,\,\,\,\, 4\quad & \quad 4 \,\,\,\,\, > \,\,\,\,\, 1\quad & \quad 8 \,\,\,\,\, < \,\,\, 10 \\
8 \,\,\,\,\, < \,\,\,\,\, 9\quad & \quad 10 \,\,\, > \,\,\,\,\, 3\quad & \quad 6 \,\,\,\,\, > \,\,\,\,\, 1\quad & \quad 8 \,\,\,\,\, > \,\,\,\,\, 5\quad & \quad 6 \,\,\,\,\, < \,\,\,\,\, 9 \\
7 \,\,\,\,\, > \,\,\,\,\, 6\quad & \quad 4 \,\,\,\,\, < \,\,\,\,\, 8\quad & \quad 5 \,\,\,\,\, > \,\,\,\,\, 2\quad & \quad 6 \,\,\,\,\, > \,\,\,\,\, 5\quad & \quad 10 \,\,\, > \,\,\,\,\, 7 \\
1 \,\,\,\,\, < \,\,\,\,\, 8\quad & \quad 3 \,\,\,\,\, < \,\,\,\,\, 9\quad & \quad 7 \,\,\,\,\, > \,\,\,\,\, 2\quad & \quad 3 \,\,\,\,\, < \,\,\, 10\quad & \quad 2 \,\,\,\,\, > \,\,\,\,\, 1 \\
8 \,\,\,\,\, > \,\,\,\,\, 2\quad & \quad 6 \,\,\,\,\, < \,\,\,\,\, 8\quad & \quad 2 \,\,\,\,\, < \,\,\,\,\, 9\quad & \quad 10 \,\,\, > \,\,\,\,\, 5\quad & \quad 5 \,\,\,\,\, > \,\,\,\,\, 4 \\
8 \,\,\,\,\, > \,\,\,\,\, 3\quad & \quad

In [20]:
s = []
for i in range(1, 6):
    s1 = []
    for j in range(10):
        s1.append("\\begin{abox}{1.2cm}" + str(10*i+j) + "\\end{abox}")
    s.append(" ".join(s1))

print("\n\n\\vspace{0.2cm}\n\n".join(s))

\begin{abox}{1.2cm}10\end{abox} \begin{abox}{1.2cm}11\end{abox} \begin{abox}{1.2cm}12\end{abox} \begin{abox}{1.2cm}13\end{abox} \begin{abox}{1.2cm}14\end{abox} \begin{abox}{1.2cm}15\end{abox} \begin{abox}{1.2cm}16\end{abox} \begin{abox}{1.2cm}17\end{abox} \begin{abox}{1.2cm}18\end{abox} \begin{abox}{1.2cm}19\end{abox}

\vspace{0.2cm}

\begin{abox}{1.2cm}20\end{abox} \begin{abox}{1.2cm}21\end{abox} \begin{abox}{1.2cm}22\end{abox} \begin{abox}{1.2cm}23\end{abox} \begin{abox}{1.2cm}24\end{abox} \begin{abox}{1.2cm}25\end{abox} \begin{abox}{1.2cm}26\end{abox} \begin{abox}{1.2cm}27\end{abox} \begin{abox}{1.2cm}28\end{abox} \begin{abox}{1.2cm}29\end{abox}

\vspace{0.2cm}

\begin{abox}{1.2cm}30\end{abox} \begin{abox}{1.2cm}31\end{abox} \begin{abox}{1.2cm}32\end{abox} \begin{abox}{1.2cm}33\end{abox} \begin{abox}{1.2cm}34\end{abox} \begin{abox}{1.2cm}35\end{abox} \begin{abox}{1.2cm}36\end{abox} \begin{abox}{1.2cm}37\end{abox} \begin{abox}{1.2cm}38\end{abox} \begin{abox}{1.2cm}39\end{abox}

\vspa

In [42]:
suanshi112ex = []

for i in range(10, 100):
    for j in range(10, 100):
        if i + j < 100:
            suanshi112ex.append(f"{i} + {j} = \dottedsquare \dottedsquare ")
            # suanshi112ex.append(f"\dottedsquare \dottedsquare + {j} = {i+j} ")
            # suanshi112ex.append(f"{i} + \dottedsquare \dottedsquare = {i+j} ")
            # suanshi111ex.append(f"{i} + \dottedsquare = {i+j} ")

suanshi112ex = np.random.permutation(suanshi112ex)

c = 0
pn = 4
nex = 40
out = []
hang = []
for s in suanshi112ex[:nex]:
    hang.append(s)
    c += 1
    if c == pn:
        out.append(" & ".join(hang) + " \\\\")
        hang = []
        c = 0
if len(hang):
    out.append("\; & \; ".join(hang) + " \\\\")
print("\n".join(out) + "\n\n a")
        

34 + 41 = \dottedsquare \dottedsquare  & 36 + 57 = \dottedsquare \dottedsquare  & 16 + 15 = \dottedsquare \dottedsquare  & 22 + 45 = \dottedsquare \dottedsquare  \\
60 + 18 = \dottedsquare \dottedsquare  & 38 + 57 = \dottedsquare \dottedsquare  & 38 + 28 = \dottedsquare \dottedsquare  & 65 + 10 = \dottedsquare \dottedsquare  \\
47 + 49 = \dottedsquare \dottedsquare  & 13 + 83 = \dottedsquare \dottedsquare  & 20 + 11 = \dottedsquare \dottedsquare  & 57 + 37 = \dottedsquare \dottedsquare  \\
18 + 59 = \dottedsquare \dottedsquare  & 75 + 14 = \dottedsquare \dottedsquare  & 16 + 63 = \dottedsquare \dottedsquare  & 57 + 38 = \dottedsquare \dottedsquare  \\
32 + 48 = \dottedsquare \dottedsquare  & 45 + 48 = \dottedsquare \dottedsquare  & 61 + 38 = \dottedsquare \dottedsquare  & 31 + 14 = \dottedsquare \dottedsquare  \\
60 + 29 = \dottedsquare \dottedsquare  & 29 + 56 = \dottedsquare \dottedsquare  & 58 + 12 = \dottedsquare \dottedsquare  & 28 + 69 = \dottedsquare \dottedsquare  \\
16 + 75 = 

In [113]:
j * t

7.2

In [109]:
lda = 5
for i in range(1, 10):
    # print(int(20*lda*np.exp(-lda)*lda**i / np.math.factorial(i)))
    # print(min(20, max(0, - 8 + 3.5 * (10 - i))))
    # print(int(max(20, min(0, - 5 + (1 + k) * i))))
    # print(255 - abs(20 - int(25*lda*np.exp(-lda)*lda**i / np.math.factorial(i))))
    print(255 - int(bo * (10 - j)) - int(min(20, max(0, - 8 + 3.5 * i)) * j * t), 255 - int(bo * (10 - j)) - int(abs(20 - 20 * lda * np.exp(-lda)*lda**i / np.math.factorial(i)) * j * t), 255 - int(bo * (10 - j)) - int(min(20, max(0, - 8 + 3.5 * (10 - i))) * j * t))

255 136 111
255 172 111
237 213 137
212 238 162
187 238 187
162 217 212
137 187 237
111 159 255
111 138 255


In [107]:
k = 1.4
t = 0.8
bo = 0.4
lda = 5
st = []
for j in range(1, 10):
    sc = [str(j)]
    for i in range(1, 10):
        nr = 255 - int(bo * (10 - j)) - int(min(20, max(0, - 8 + 3.5 * i)) * j * t)
        ng = 255 - int(bo * (10 - j)) - int(abs(20 - 20 * lda * np.exp(-lda)*lda**i / np.math.factorial(i)) * j * t)
        nb = 255 - int(bo * (10 - j)) - int(min(20, max(0, - 8 + 3.5 * (10 - i))) * j * t)
        bulin = ""
        if i * j < 10:
            bulin = "\\phantom{0}"
        sc.append("\\cellcolor[RGB]{"+f"{nr},{ng},{nb}"+"}"+bulin+str(i * j))
    st.append(" & ".join(sc) + "\\\\\n")
print("\n\n"+"\hline\n".join(st)+"\\hline")



1 & \cellcolor[RGB]{252,239,236}\phantom{0}1 & \cellcolor[RGB]{252,243,236}\phantom{0}2 & \cellcolor[RGB]{250,248,239}\phantom{0}3 & \cellcolor[RGB]{248,251,242}\phantom{0}4 & \cellcolor[RGB]{245,251,245}\phantom{0}5 & \cellcolor[RGB]{242,248,248}\phantom{0}6 & \cellcolor[RGB]{239,245,250}\phantom{0}7 & \cellcolor[RGB]{236,242,252}\phantom{0}8 & \cellcolor[RGB]{236,239,252}\phantom{0}9\\
\hline
2 & \cellcolor[RGB]{252,226,220}\phantom{0}2 & \cellcolor[RGB]{252,234,220}\phantom{0}4 & \cellcolor[RGB]{248,243,226}\phantom{0}6 & \cellcolor[RGB]{243,249,232}\phantom{0}8 & \cellcolor[RGB]{237,249,237}10 & \cellcolor[RGB]{232,244,243}12 & \cellcolor[RGB]{226,237,248}14 & \cellcolor[RGB]{220,231,252}16 & \cellcolor[RGB]{220,226,252}18\\
\hline
3 & \cellcolor[RGB]{253,214,205}\phantom{0}3 & \cellcolor[RGB]{253,226,205}\phantom{0}6 & \cellcolor[RGB]{247,239,214}\phantom{0}9 & \cellcolor[RGB]{239,248,222}12 & \cellcolor[RGB]{231,248,231}15 & \cellcolor[RGB]{222,241,239}18 & \cellcolor[RGB]{214,

In [142]:
st = []
for j in range(1, 10):
    sc = []
    for i in range(1, 10):
        de = ""
        bulin = ""
        ji = i * j
        ji1 = ji % 10
        ji2 = ji // 10
        ji2_str = 数转中[ji2] + "十"
        if ji2 < 1:
            de = "得"
            ji2_str = ""
        # if ji2 < 2:
        #     ji2_str = "十"
        #     bulin =  "\\phantom{十}"
        # else:
        ji1_str = 数转中[ji1]
        if ji1 < 1:
            ji1_str = ""
        txt = f"{数转中[j]}{数转中[i]}{de}{ji2_str}{ji1_str}{bulin}"
        sc.append("\\texttt{" + txt + "}")
    st.append("    " + " & ".join(sc[:5]) + "\\\\")
    st.append("    " + " & ".join(sc[5:]) + " & \\\\")

ncol = 5
nem = 5
header = ["p{"+str(nem)+"em}"] * 9
print("{ " + " ".join(header) + "}\n"+"\n".join(st) + "\n\n")

{ p{5em} p{5em} p{5em} p{5em} p{5em} p{5em} p{5em} p{5em} p{5em}}
    \texttt{一一得一} & \texttt{一二得二} & \texttt{一三得三} & \texttt{一四得四} & \texttt{一五得五}\\
    \texttt{一六得六} & \texttt{一七得七} & \texttt{一八得八} & \texttt{一九得九} & \\
    \texttt{二一得二} & \texttt{二二得四} & \texttt{二三得六} & \texttt{二四得八} & \texttt{二五一十}\\
    \texttt{二六一十二} & \texttt{二七一十四} & \texttt{二八一十六} & \texttt{二九一十八} & \\
    \texttt{三一得三} & \texttt{三二得六} & \texttt{三三得九} & \texttt{三四一十二} & \texttt{三五一十五}\\
    \texttt{三六一十八} & \texttt{三七二十一} & \texttt{三八二十四} & \texttt{三九二十七} & \\
    \texttt{四一得四} & \texttt{四二得八} & \texttt{四三一十二} & \texttt{四四一十六} & \texttt{四五二十}\\
    \texttt{四六二十四} & \texttt{四七二十八} & \texttt{四八三十二} & \texttt{四九三十六} & \\
    \texttt{五一得五} & \texttt{五二一十} & \texttt{五三一十五} & \texttt{五四二十} & \texttt{五五二十五}\\
    \texttt{五六三十} & \texttt{五七三十五} & \texttt{五八四十} & \texttt{五九四十五} & \\
    \texttt{六一得六} & \texttt{六二一十二} & \texttt{六三一十八} & \texttt{六四二十四} & \texttt{六五三十}\\
    \texttt{六六三十六} & \texttt{六七四十二} & \texttt{六八四十八}

In [154]:
x1 = 1
x2 = 2
bar = 0.02
f = lambda x : x * x * x - 2

lb = x1
ub = x2
for i in range(10):
    mid = (lb + ub) / 2
    if f(mid) < 0:
        lb = mid
    else:
        ub = mid
    if ub - lb < bar:
        break

r = 2 ** (1/3)
mid = (lb + ub) / 2
t = (r - lb) / (ub - lb) * 100
print(i, lb, ub, mid, f"{r:0.4f}", f"{t:0.2f}%")

5 1.25 1.265625 1.2578125 1.2599 63.49%


In [ ]:
y = x^3 - 2
y = 3a^2 (x - a) + a^3 - 2 = 0
x = a - (a^3 - 2) / 3a^2
  = (3a^3 - a^3 + 2) / 3a^2
  = 2(a^3 + 1) / 3a^2

In [156]:
nt = lambda x : 2 * (x ** 3 + 1) / (3 * x ** 2)

x_0 = 2
a = x_0
for i in range(10):
    print(i, a)
    a = nt(a)

0 2
1 1.5
2 1.2962962962962963
3 1.2609322247417485
4 1.2599218605659261
5 1.2599210498953948
6 1.259921049894873
7 1.2599210498948732
8 1.259921049894873
9 1.2599210498948732


In [159]:
abs(2 ** (1/3) - 1.2599218605659261) / 2 ** (1/3) * 1000000

0.6434300411290248

In [172]:
def dint(a, b, f, ns):
    if not hasattr(ns, "__iter__"):
        ns = [ns]
    # 定积分
    ss = []
    for n in ns:
        dt = (b - a) / n
        s = 0
        for i in range(n):
            s += f(a + i*dt) * dt
        ss.append(s)
    if len(ss) == 1:
        ss = ss[0]
    return np.array(ss)

a = 0
b = 6
f = lambda x: 4 + np.sin(2 + x + np.sqrt(x)) * 0.5 + 0.4 * x - 0.2 * np.cos(0.7 * x)
n = [1000, 2000, 10000, 20000, 50000, 100000, 200000]

dint(a, b, f, n) / (b - a)


array([5.23931447, 5.23976633, 5.24012848, 5.24017381, 5.24020103,
       5.2402101 , 5.24021464])

In [184]:
a = 160
b = 60
r = 240

4*a*b , 2 * a * b * (a + b), a ** 2 * b ** 2
# r ** 4 - 4*a*b * r ** 2 - 2 * a * b * (a + b) * r - a ** 2 * b ** 2

(38400, 4224000, 92160000)

In [191]:
# a^4 - 38400a^2 - 4224000 a - 92160000
# 4a^3 - 76800 a - 4224000
f = lambda a: a ** 3 - 19200 * a - 1056000
g = lambda a: a ** 2 - 6400
f(200)

3104000

In [258]:
def horner(a, x):
    n = len(a)
    b = np.zeros_like(a, dtype=float)
    t = 0
    for i in range(n):     
        t = a[i] + t * x
        # print(i, t)   
        b[i] = t
    # print(b)
    return b[-1], b[:-1]

horner(np.array([0]), 1)

(0.0, array([], dtype=float64))

In [266]:
def NewtonHorner(a, x0=0, tol=0.0001, nmax=100, aim="all"):
    n = len(a) - 1
    roots = np.zeros((n, 1))
    iter = np.zeros((n, 1))
    rest = []

    i = 0
    while i < n:
        j = 0
        xn = x0
        diff = tol + 1
        while j < nmax and abs(diff) >= tol:
            f, b = horner(a, xn)
            fp, _ = horner(b, xn)
            diff = - f / fp
            xn += diff
            j += 1

        if j == nmax:
            print(f'Fail to converge in search of root no. {i+1} within {tol:0.4f} after {nmax} iterations.')
            print(f"From {x0:0.3f} to {xn:0.3f}, poly value: {f:0.3f}")
            rest = a
            break

        if aim == "one":
            return xn, j
        
        v = 0
        while abs(v) < tol:
            v, _ = horner(a, xn)
            if abs(v) < tol:
                print(f"not bad root {i+1}...")
                v, a = horner(a, xn)
                roots[i] = xn
                iter[i] = j
                i += 1
        if len(a) == 1:
            break

    
    return roots, iter, rest

a = [1, 0, -38400, -4224000, -92160000]
# a = [1, -3, 3, -1, 0, 0]
roots, iter, rest = NewtonHorner(a, x0=0, nmax=1000)
roots, iter, rest

not bad root 1...
not bad root 2...
Fail to converge in search of root no. 3 within 0.0001 after 1000 iterations.
From 0.000 to -153.931, poly value: 14935.466


(array([[-29.603158],
        [240.      ],
        [  0.      ],
        [  0.      ]]),
 array([[ 5.],
        [18.],
        [ 0.],
        [ 0.]]),
 array([1.00000000e+00, 2.10396842e+02, 1.29715890e+04]))

In [345]:
m = 0.1 / 1000
g = 9.8
v_0 = 0.2
b = 0.0003
h = 2000

m*g, m*g / b, m * g / b / b, v_0 / b, m * g / b / b - v_0 / b

(0.0009800000000000002,
 3.2666666666666675,
 10888.888888888892,
 666.6666666666667,
 10222.222222222226)

In [348]:
from scipy.optimize import fsolve

f = lambda t: m*g / b * t + (np.exp(-b * t) - 1) * (m * g / b / b - v_0 / b) - h
t_inf = fsolve(f, h)[0]
t_inf

2049.3809204566173

In [349]:
v = lambda t: m*g / b * (1 - np.exp( -b * t)) + v_0 * np.exp( -b * t)

v(t_inf), np.exp( -b * t_inf)

(1.6083933020474852, 0.5407413145497332)